In [3]:
from catboost import CatBoostRegressor


In [4]:

# Initialize data
train_data = [[1, 4, 5, 6], [4, 5, 6, 7], [30, 40, 50, 60]]

eval_data = [[2, 4, 6, 8], [1, 4, 50, 60]]

train_labels = [10, 20, 30]
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=2, learning_rate=1, depth=2)
# Fit model
model.fit(train_data, train_labels)
# Get predictions
preds = model.predict(eval_data)
print(preds)


0:	learn: 6.1237244	total: 68.5ms	remaining: 68.5ms
1:	learn: 4.5927933	total: 69ms	remaining: 0us
[15.625 18.125]


## Save Model
Model must have the file name `model.pkl` in current directory. This is hardcoded in `deploy.sh`

In [5]:
# save model
import pickle
with open('model.pkl', 'wb') as out:
    pickle.dump(model, out, protocol=3) # python3 default
  

## Deploy Endpoint

Take note that the following needs to be updated according:
- image name
- classic model location
- sagemaker execution role

In [47]:
#! cp model.pkl local_test/test_dir/model/

In [23]:
!./deploy.sh sagemaker-inference-containers/rt-c360

a model.pkl
move: ./model.tar.gz to s3://wy-sandbox-singapore/lease-plan/classic-model/sagemaker-inference-containers/rt-c360/input/model.tar.gz
342474125894
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
Sending build context to Docker daemon  265.2kB
Step 1/10 : FROM 520713654638.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-tensorflow-scriptmode:1.12-cpu-py3
 ---> ba542f0b9706
Step 2/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 27933e9ed3c7
Step 3/10 : RUN pip3 install --upgrade pip
 ---> Using cache
 ---> d10eabcef50a
Step 4/10 : RUN pip3 install --no-cache --upgrade --use-feature=2020-resolver     flask gevent gunicorn     numpy==1.19.1     pandas==1.0.5     scikit-learn==0.23.2     requests==2.24.0     scipy==1.5.1     catboost==0.24.1
 ---> Using cache
 ---> e4c95db3709b
Step 5/10 : ENV PYTHONUNBUF

In [27]:
!aws sagemaker list-endpoints --name-contains rtc360-20200929-214740

{
    "Endpoints": [
        {
            "EndpointName": "rtc360-20200929-214740",
            "EndpointArn": "arn:aws:sagemaker:ap-southeast-1:342474125894:endpoint/rtc360-20200929-214740",
            "CreationTime": 1601387263.274,
            "LastModifiedTime": 1601387696.114,
            "EndpointStatus": "InService"
        }
    ]
}


In [26]:
!date

Tue Sep 29 21:49:38 +08 2020


## Inference


In [28]:
!echo "1,2,3,4\n5,6,7,8" >> test.csv

In [29]:
with open('test.csv', 'r') as f:
    payload = f.read()
    
print(payload)

1,2,3,4
5,6,7,8
1,2,3,4
5,6,7,8



In [31]:
# leaseplan-deploy-py3-20191015-055338 (iso match sklearn)
# leaseplan-deploy-py3-20191016-083601 (v2)

import io, boto3

sess = boto3.Session()
response = sess.client('sagemaker-runtime').invoke_endpoint(
    EndpointName='rtc360-20200929-214740', 
    Body=payload,
    ContentType='text/csv', 
)

results = response['Body'].read().decode('ascii')
print(results)


15.625
20.0
15.625
20.0

